# model.fit() vs. tf.GradientTape()

케라스의 model.fit()과 Gradient Tape()를 사용한 구현의 차이를 이해해봅시다.

## 1. model.fit()

In [ ]:
'''
# 신경망 모델 만들기
model = tf.keras.models.Sequential()
# 완전 연결층을 추가
model.add(tf.keras.layers.Dense(1))
# 옵티마이저와 손실 함수를 지정합니다.
model.compile(optimizer = 'sgd', loss = 'mse')
# 훈련 데이터를 사용하여 에포크 횟수만큼 훈련
model.fit(x_train, y_train, epochs = 10)
'''

"\n# 신경망 모델 만들기\nmodel = tf.keras.models.Sequential()\n# 완전 연결층을 추가\nmodel.add(tf.keras.layers.Dense(1))\n# 옵티마이저와 손실 함수를 지정합니다.\nmodel.compile(optimizer = 'sgd', loss = 'mse')\n# 훈련 데이터를 사용하여 에포크 횟수만큼 훈련\nmodel.fit(x_train, y_train, epochs = 10)\n"

위 코드를 조금 복잡하게 작성하면 아래와 같다.

## 2. tf.GradientTape()

tape_gradient() 메서드는 자동 미분 기능을 수행합니다.  
자동 미분에 대해서 실습을 통해 이해해봅시다. 임의로 2w^2+5라는 식을 세워보고, w에 대해 미분해보겠습니다.

In [ ]:
w = tf.Variable(2.)

def f(w):
  y = w**2
  z = 2*y + 5
  return z

이제 gradients를 출력하면 w가 속한 수식을 w로 미분한 값이 저장된 것을 확인할 수 있습니다.

In [ ]:
with tf.GradientTape() as tape:
  z = f(w)

gradients = tape.gradient(z, [w])
print(gradients)

[<tf.Tensor: shape=(), dtype=float32, numpy=8.0>]


In [ ]:
'''
# 훈련할 가중치 변수를 선언
w = tf.Variable(tf.zeros(shape=(1)))
b = tf.Variable(tf.zeros(shape=(1)))

# 경사 하강법 옵티마이저 설정
optimizer = tf.optimizer.SGD(lr = 0.01)
# 에포크만큼 훈련
num_epochs = 10
for step in range(num_epochs):
   
    # 예측을 해서 손실을 구하는 과정입니다. (자동 미분을 위해 연산 과정을 기록합니다.)
    # tape_gradient() 메서드를 사용하면 그래디언트를 자동으로 계산할 수 있도록 합니다.
    with tf.GradientTape() as tape:
        z_net = w * x_train + b # 정방향 계산
        z_net = tf.reshape(z_net, [-1])
        sqr_errors = tf.square(y_train - z_net)
        mean_cost = tf.reduce_mean(sqr_errors) # 손실을 계산

    # 경사하강법으로 파라미터를 업데이트하는 과정입니다.
    # 1. 가중치에 대한 그래디언트 계산
    grads = tape.gradient(mean_cost, [w, b])

    # 2. 가중치를 업데이트
    # apply_gradients() 메서드에는 그래디언트와 가중치를 튜플로 묶은 리스트를 전달해야 합니다.
    # 보통 zip()을 주로 사용합니다.
    optimizer.apply_gradient(zip(grads, [w, b]))
'''

'\n# 훈련할 가중치 변수를 선언\nw = tf.Variable(tf.zeros(shape=(1)))\nb = tf.Variable(tf.zeros(shape=(1)))\n\n# 경사 하강법 옵티마이저 설정\noptimizer = tf.optimizer.SGD(lr = 0.01)\n# 에포크만큼 훈련\nnum_epochs = 10\nfor step in range(num_epochs):\n   \n    # 예측을 해서 손실을 구하는 과정입니다. (자동 미분을 위해 연산 과정을 기록합니다.)\n    # tape_gradient() 메서드를 사용하면 그래디언트를 자동으로 계산할 수 있도록 합니다.\n    with tf.GradientTape() as tape:\n        z_net = w * x_train + b # 정방향 계산\n        z_net = tf.reshape(z_net, [-1])\n        sqr_errors = tf.square(y_train - z_net)\n        mean_cost = tf.reduce_mean(sqr_errors) # 손실을 계산\n\n    # 경사하강법으로 파라미터를 업데이트하는 과정입니다.\n    # 1. 가중치에 대한 그래디언트 계산\n    grads = tape.gradient(mean_cost, [w, b])\n\n    # 2. 가중치를 업데이트\n    # apply_gradients() 메서드에는 그래디언트와 가중치를 튜플로 묶은 리스트를 전달해야 합니다.\n    # 보통 zip()을 주로 사용합니다.\n    optimizer.apply_gradient(zip(grads, [w, b]))\n'